# Extract Time Series Sentinel-2 data
## from Digital Earth Australia (DEA) via STAC

In [1]:
!python --version

Python 3.8.13


In [2]:
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

import pystac_client
import odc.stac
# odc-stac library downloads DEA datasets stored in AWS
# when external to AWS (like outside DEA sandbox), AWS signed requests must be disabled
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

from datacube.utils.geometry import point, box, CRS, Geometry, Coordinate, BoundingBox, GeoBox
from datacube.model import GridSpec
from affine import Affine

# My helper class
from helperfunctions import begin_timer, end_timer, saveDataset, loadDataset
from dea_helperfunctions import DEA_HelperFunctions

from dea_tools.plotting import rgb
from dea_tools.bandindices import calculate_indices

# Set some configurations for displaying tables nicely
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

### Constants

In [3]:
# configure helper functions
dea_fns = DEA_HelperFunctions()
basepath = "../datasets/"
fileextn = ".pkl"

filespecifier = "2017-float32/"

# Survey Area origin point
survey_origin = [138.43196647747274, -34.62929501472954 ] # PortGawlerBeach, SA

# Test
#timebands = [["2016-07-01","2016-07-05"], ["2017-07-01","2017-07-05"], ["2018-07-01","2018-07-05"]]

# Jan to Dec 2016
timebands = [["2016-01-01","2016-01-31"], ["2016-02-01","2016-02-29"], ["2016-03-01","2016-03-31"],
             ["2016-04-01","2016-04-30"], ["2016-05-01","2016-05-31"], ["2016-06-01","2016-06-30"],
             ["2016-07-01","2016-07-31"], ["2016-08-01","2016-08-31"], ["2016-09-01","2016-09-30"],
             ["2016-10-01","2016-10-31"], ["2016-11-01","2016-11-30"], ["2016-12-01","2016-12-31"]]

# Jan to Dec 2017
#timebands = [["2017-01-01","2017-01-31"], ["2017-02-01","2017-02-28"], ["2017-03-01","2017-03-31"],
#             ["2017-04-01","2017-04-30"], ["2017-05-01","2017-05-31"], ["2017-06-01","2017-06-30"],
#             ["2017-07-01","2017-07-31"], ["2017-08-01","2017-08-31"], ["2017-09-01","2017-09-30"],
#             ["2017-10-01","2017-10-31"], ["2017-11-01","2017-11-30"], ["2017-12-01","2017-12-31"]]

# Coordinate Reference Systems (CRS)
# !! ENSURE YOU USING CONSISTENT CRS WHEN PLOTTING AREAS OR PERFORMING GEOMETRIC OPERATIONS.
epsg4326 = 'epsg:4326'            # EPSG:4326 | WGS84 latitude-longitude CRS | in Degrees of Latitude and Longitude
epsg3577 = 'epsg:3577'            # EPSG:3577 | GDA94 / Australian Albers projection | in Metres from CRS Centroid

# Survey Area (Region of Interest)
resolution_int = 10 # where each pixel is of 10m x 10m resolution
resolution = (-resolution_int,resolution_int) # where each pixel is of 10m x 10m resolution
survey_tilesize = (5120,5120)     # size of each tile in metres
survey_tiles = (10,5)              # number of horizontal and vertical tiles change to 10,5 for final run

survey_tilesize_pixels = tuple(int(ti/resolution_int) for ti in survey_tilesize)
print(survey_tilesize_pixels)

# Satellite datasets
collections = ["s2a_ard_granule","s2b_ard_granule"] # Sentinel-2A and 2B MSI Definitive ARD - NBART and Pixel Quality
bands = ("nbart_blue", "nbart_green", "nbart_red", "nbart_nir_1", "fmask") # Satellite Bands

# The following band indices are added to the datasets
#        'NDVI' (Normalised Difference Vegetation Index, Rouse 1973)
#        'NDWI' (Normalised Difference Water Index, McFeeters 1996)
#        'kNDVI' (Non-linear Normalised Difference Vegetation Index, Camps-Valls et al. 2021)
#         Note: kNDVI is more resistant to saturation, bias, and complex phenological cycles
#               and shows enhanced robustness to noise and stability across spatial and temporal scales.
additional_bands = ['NDVI','kNDVI','NDWI']

(512, 512)


### Define and Display Survey Area Grid Tiles

In [4]:
# Subdivide survey area into tiles
surveyarea_polygons = dea_fns.calc_surveyarea_polygons( origin=survey_origin,
                                                       tileresolution=survey_tilesize,
                                                       numtiles=survey_tiles)
# Add polygons to geodataframe
gdf_surveyarea = gpd.GeoDataFrame(columns=["tile", "geometry"], crs=epsg4326)

gdf_surveyarea['geometry'] = surveyarea_polygons
gdf_surveyarea['tile'] = range(1, len(gdf_surveyarea) + 1)
gdf_surveyarea.to_crs(epsg3577, inplace=True)

# Save data set to pickle file
saveDataset(basepath + filespecifier + "tilepolygons" + fileextn, gdf_surveyarea)

# display Tile areas
gdf_surveyarea.head(5)

,tile,geometry
0,1,"POLYGON ((587352.715 -3795648.621, 587610.746 -3790527.770, 592718.624 -3790786.270, 592458.349 -3795907.007, 587352.715 -3795648.621))"
1,2,"POLYGON ((592458.349 -3795907.007, 592718.624 -3790786.270, 597826.388 -3791047.006, 597563.870 -3796167.629, 592458.349 -3795907.007))"
2,3,"POLYGON ((597563.870 -3796167.629, 597826.388 -3791047.006, 602934.037 -3791309.980, 602669.277 -3796430.487, 597563.870 -3796167.629))"
3,4,"POLYGON ((602669.277 -3796430.487, 602934.037 -3791309.980, 608041.570 -3791575.191, 607774.568 -3796695.581, 602669.277 -3796430.487))"
4,5,"POLYGON ((607774.568 -3796695.581, 608041.570 -3791575.191, 613148.987 -3791842.639, 612879.742 -3796962.912, 607774.568 -3796695.581))"


In [5]:
map_zoom_level = 9
map_centroid = survey_origin.reverse() # swap lat and long

m = gdf_surveyarea.explore(
     column="tile", # make choropleth based on Commodity description
     tooltip=["tile"], popup=True, # show all values in popup (on click)
     tiles="OpenStreetMap", # use "CartoDB positron" tiles
     cmap="Wistia", # use "Set1" matplotlib colormap
     legend=False,
     location=map_centroid,
     zoom_start = map_zoom_level,
     name='Survey Area Tiles') # use black outline

folium.TileLayer('CartoDB positron', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m  # show map

### Open Catalog & Extract images from Survey area

In [6]:
catalog = pystac_client.Client.open('https://explorer.sandbox.dea.ga.gov.au/stac')

# Iterate through each GeoDataFrame polygon
# Extract bands for each polygon for each time period specified

surveyarea_ds = []
total_durn = 0
total_numdatasets = 0

for t in timebands:
    timeband_numdatasets = 0
    stime = begin_timer(info=False)
    
    #i=0
    
    datem = datetime. datetime. strptime(t[0], "%Y-%m-%d")
    y = datem.year # year
    m = datem.month # month
    print(' ■', end='')
    
    #for p in surveyarea_polygons:
    for index, gdf_sa in gdf_surveyarea.iterrows():
        print('□', end='')
        
        #i+=1
        #bb_4326 = dea_fns.convert_poly2bbox(p)
        #b_4326 = dea_fns.convert_poly2box(p)
        #b = b_4326.to_crs(epsg3577)
        #bb = b.boundingbox
        
        i = int(gdf_sa[0])
        p = gdf_sa[1]
      
        b = dea_fns.convert_poly2box(p,crs=CRS('epsg:3577'))
        b_4326 = b.to_crs(epsg4326)
        
        bb = b.boundingbox
        bb_4326 = b_4326.boundingbox
        
        
        query = catalog.search( bbox=bb_4326, collections=collections, datetime=f"{t[0]}/{t[1]}" )
        items = list(query.get_items()) # Search the STAC catalog
        timeband_numdatasets += len(items)
        
        #print(f"Time band: {t[0]} to {t[1]} | Tile #: {i} | Found: {len(items):d} datasets")
        #print(bb_4326)
        #print("---------------------------------------------------------------")
        
        if len(items) > 0:
            surveyarea_affine = Affine(resolution[1], 0.0, bb.left, 0.0, resolution[0], bb.top) ##############
            #surveyarea_affine = Affine(resolution[1], 0.0, bb.left, 0.0, resolution[0], bb.bottom) ##############
            try:
                ds = odc.stac.load( items, bands=bands,
                                geobox=GeoBox(survey_tilesize_pixels[0], # width in pixels
                                              survey_tilesize_pixels[1], # height in pixels
                                              surveyarea_affine,
                                              epsg3577), # Output Coordinate Reference System (CRS)
                                groupby="solar_day")
                calculate_indices(ds=ds, index=additional_bands, collection='ga_s2_1', inplace=True)
                
                # Convert float64 layers to float32 for indices to save space
                dscopy = ds.copy(deep=True)
                dscopy['NDVI'] = ds['NDVI'].astype(dtype='float32', casting='same_kind', copy=True)
                dscopy['kNDVI'] = ds['kNDVI'].astype(dtype='float32', casting='same_kind', copy=True)
                dscopy['NDWI'] = ds['NDWI'].astype(dtype='float32', casting='same_kind', copy=True)
                dscopy['fmask'] = ds['fmask'].astype(dtype='uint8', casting='same_kind', copy=True)
                # Convert y and x after all other layers to else other layers populated iwth NaN
                dscopy['y'] = ds['y'].astype(dtype='float32', casting='same_kind', copy=True)
                dscopy['x'] = ds['x'].astype(dtype='float32', casting='same_kind', copy=True)
                
                surveyarea_ds.append([t, y, m, len(items), i, bb, dscopy])
            except Exception as e:
                print(f"Could not download TIFF image from DEA due to {e}.")

    etime, durn = end_timer(stime, info=False)
    print(f' Duration(s): {durn:9.3f} | Total Dataset(s): {timeband_numdatasets:d}')
    total_durn += durn
    total_numdatasets += timeband_numdatasets
    
print(f'Total Duration(s): {total_durn:9.3f} | Total Dataset(s): {total_numdatasets:d}')

# Save statistics to txt file
original_stdout = sys.stdout # Save a reference to the original standard output
with open(basepath + filespecifier + '_log.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(f'DEA_ExtractSentinel | {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
    print(f'Total Duration(s): {total_durn:9.3f} | Total Dataset(s): {total_numdatasets:d}')
    sys.stdout = original_stdout # Reset the standard output to its original value
    
# Save data set to pickle file
saveDataset(basepath + filespecifier + "initialdataset" + fileextn, surveyarea_ds)

 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   508.031 | Total Dataset(s): 388
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   358.177 | Total Dataset(s): 273
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   207.766 | Total Dataset(s): 158
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   392.905 | Total Dataset(s): 309
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   557.083 | Total Dataset(s): 395
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   392.500 | Total Dataset(s): 309
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   590.414 | Total Dataset(s): 431
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   521.116 | Total Dataset(s): 388
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   641.347 | Total Dataset(s): 440
 ■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ Duration(s):   561.402 | Total Dataset(s): 388


In [7]:
from objectfunctions import total_size, total_memsize
print(f'Total size of surveyarea_ds: {total_memsize(surveyarea_ds)/1024/1024:0.3f} Mb')

Total size of surveyarea_ds: 15747.293 Mb
